In [16]:
from llm_init import llm

In [10]:
from typing_extensions import TypedDict, Literal, Dict, Optional, List
from langchain_core.prompts import ChatPromptTemplate

class OverallState(TypedDict):
    user_input: str
    preferences: Optional[Dict[str, str]]
    recommended_listing: List[str]
    graph_output: str


In [11]:
from prompt import get_system_prompt
from langgraph.types import interrupt, Command
from langgraph.graph import StateGraph, START, END

class Preference(TypedDict):
    buy_rent: Optional[Literal['Buy', 'Rent']]
    location: Optional[str]
    size: Optional[str]
    business_nature: Optional[str]
    power_requirement: Optional[str]
    floor_loading_capacity: Optional[str]
    ceiling_height: Optional[str]
    budget: Optional[str]

class Planner(TypedDict):
    planner_decision: Literal['final_output', 'property_listing_lookup']
    rephrased_query: str
    preferences: Preference
    messages: str

def Landy_Planner(OverallState):  
    user_input = OverallState['user_input']  

    system_prompt = get_system_prompt()

    template = ChatPromptTemplate(
        [
            ("system", system_prompt),
            ("human", "{user_input}"),
        ]
    )

    chain = template | llm.with_structured_output(Planner)

    response = chain.invoke({
        "user_input": user_input
    })

    if response['planner_decision'] == 'final_output':
        goto = END
    else:
        goto = "Property_Lookup_Agent"
    
    return Command(
        update = {
        'graph_output': response['messages'],
        'preferences': response['preferences']
        },
        goto = goto
        )

In [12]:
from prompt import get_property_search_prompt
import json

class Listing(TypedDict):
    location: Optional[str]
    built_up_area_sqft: Optional[str]
    land_area_sqft: Optional[str]
    zoning_type: Optional[str]
    power_capacity_kva: Optional[str]
    floor_loading_ton_per_sqm: Optional[str]
    clear_height_m: Optional[str]
    asking_price: Optional[str]
    source: Optional[str]
    
class FinalOutput(TypedDict):
    recommended_listing = Optional[List[Listing]]
    messages: str

# Open the file in read mode ('r')
with open('listing.json', 'r') as file:
    # Use json.load() to convert the file content to a Python dictionary
    listing = json.load(file)

def Property_Lookup_Agent(state):    
    property_search_prompt = get_property_search_prompt()

    template = ChatPromptTemplate(
        [
            ("system", property_search_prompt),
            ("human", "{user_preferences}"),
        ]
    )

    chain = template | llm.with_structured_output(FinalOutput)
    response = chain.invoke({
        "listing": listing,
        "user_preferences": OverallState['preferences'],
    })

    print(response.get('recommended_listing', ''))
    
    return Command(
        update = {
            'graph_output': response['messages'],
            'recommended_listing': response['recommended_listing']
            },
        goto = END)

In [13]:

builder = StateGraph(OverallState)
builder.add_node("Landy_Planner", Landy_Planner)
builder.add_node("Property_Lookup_Agent", Property_Lookup_Agent)

builder.add_edge(START, "Landy_Planner")
builder.add_edge("Property_Lookup_Agent", END)

graph = builder.compile()


In [15]:
graph.invoke({"user_input": "Hi"})


{'user_input': 'Hi',
 'preferences': {'buy_rent': '',
  'location': '',
  'size': '',
  'business_nature': '',
  'power_requirement': '',
  'floor_loading_capacity': '',
  'ceiling_height': '',
  'budget': ''},
 'graph_output': 'Hello! How can I assist you in finding an industrial property in the Klang Valley today? Are you looking to buy or rent, and do you have any specific requirements in mind such as location, size, or budget?'}